# Biliteracy Seal

In this Jupyter notebook students' eligibility for the State Seal of Biliteracy will be calculated by loading in the cleaned transcript file for Class Grade Corrections for ILP as well as the state assessment data for SBAC and the most recent ELPAC for students that are still English Language Learners. The students GPA in their Spanish courses will be calculated, the GPA for their top four ELA classes will be calculated, whether they Met the SBAC or not will be looked at, and if they are an EL students it will look at their performance on the Oral part of the test. Students will be filtered down to just the Spanish 3 students and they will then be looked at if they met the English requirement (3.0 GPA or SBAC Met), Spanish requirement (3.0 GPA) and if they are EL they met on the Oral portion of ELPAC.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# LIST STU SC ID CID LN FN GR LF 
roster = pd.read_excel(r"C:\Users\derek.castleman\Desktop\Roster Biliteracy.xlsx")

# From Course Correction for ILP Jupyter notebook
transcripts = pd.read_csv(r"C:\Users\derek.castleman\Desktop\Fixed Transcripts.csv")

# Raw Cers file for state tests
tests = pd.read_csv(r"C:\Users\derek.castleman\Desktop\Joie Stuff\CERS_SBAC_2023-24_6_20_24ALL.csv")

# LIST STU ID TST TST.ID TST.PT TST.PL TST.TD 
elpac = pd.read_excel(r"C:\Users\derek.castleman\Desktop\elpac.xlsx")

In [ ]:
transcripts

## Spanish

The first part will filter the Spanish courses out of the transcripts and calculate the GPA for those courses that the students have taken.

In [ ]:
# Selecting foreing language classes
spanish = transcripts[transcripts['CSU Req'] == 'E']
spanish

In [ ]:
# Selecting out any that are Spanish
spanish = spanish[spanish['Course'].str.contains('spa', case=False, na=False)]
spanish

### BC Spanish Courses

The BC Spanish course actually counts as two grades since it is a year class. The BC courses will be selected out and then duplicated to represent this full year representation.

In [ ]:
# Selecting the non-bc courses
non_bc = spanish[spanish['Crs Lvl'] != 24]
non_bc

In [ ]:
# Selecting the BC courses
bc = spanish[spanish['Crs Lvl'] == 24]
bc

In [ ]:
# Doubling the BC courses
bc = pd.concat([bc, bc])
bc

In [ ]:
# Adding the Non-BC to the BC to get all classes again
spanish = pd.concat([non_bc, bc])
spanish

### Spanish GPA

In this section the GPA for all of the Spanish classes taken will be calculated. It will be done by filtering them out by A's (a point value of 4, B's (value of 3), C's (value of 2), D's (value of 1), and F's with no value. The GPA will be calculated by dividing the total points earned by the number of semesters that a student has taken.

In [ ]:
# Gives a value of 1 for each semester taken
spanish['Spanish Semesters'] = 1
spanish

In [ ]:
# A grades are chosen
spanish_a = spanish[(spanish['Mark'] == 'A+') | (spanish['Mark'] == 'A') | (spanish['Mark'] == 'A-')]
spanish_a

In [ ]:
# A value of 4 is given to A
spanish_a['Points'] = 4
spanish_a

In [ ]:
# The B grades are chosen
spanish_b = spanish[(spanish['Mark'] == 'B+') | (spanish['Mark'] == 'B') | (spanish['Mark'] == 'B-')]
spanish_b

In [ ]:
# A value of 3 is given to Bs
spanish_b['Points'] = 3
spanish_b

In [ ]:
# The C grades are selected
spanish_c = spanish[(spanish['Mark'] == 'C+') | (spanish['Mark'] == 'C') | (spanish['Mark'] == 'C-')]
spanish_c

In [ ]:
# A value of 2 is given to C grades
spanish_c['Points'] = 2
spanish_c

In [ ]:
# The D grades are selected
spanish_d = spanish[(spanish['Mark'] == 'D+') | (spanish['Mark'] == 'D') | (spanish['Mark'] == 'D-')]
spanish_d

In [ ]:
# A value of 1 is given to D
spanish_d['Points'] = 1
spanish_d

In [ ]:
# The F grades are selected
spanish_f = spanish[spanish['Mark'] == 'F']
spanish_f

In [ ]:
# A value of 0 is given to F
spanish_f['Points'] = 0
spanish_f

In [ ]:
# All the grades are put back together again
spanish = pd.concat([spanish_a, spanish_b, spanish_c, spanish_d, spanish_f])
spanish

In [ ]:
# The sum of semesters taken is calculated
spanish_courses = spanish.groupby(['Student ID'])['Spanish Semesters'].sum().reset_index()
spanish_courses

In [ ]:
# The sum of points is added up based on the grades
spanish_points = spanish.groupby(['Student ID'])['Points'].sum().reset_index()
spanish_points

In [ ]:
# Semester and points are put together
spanish = pd.merge(spanish_courses, spanish_points, how='inner', on='Student ID')
spanish

In [ ]:
# GPA is calculated by dividng the points by semesters completed
spanish['Spanish GPA'] = spanish['Points'] / spanish['Spanish Semesters']
spanish

In [ ]:
# Just the columns of interest are selected
spanish = spanish[['Student ID', 'Spanish Semesters', 'Spanish GPA']]
spanish

## ELA

The GPA for students in their ELA courses will be caclulated in the same manner as the Spanish GPA. But since it is only the ones required for graduation, it will be the top four courses that a student has completed that will be used in calculating the GPA. This will be done by summing the points earned in each course and then have them in descending order and selecting the top four.

In [ ]:
# ELA courses are selected by UC/CSU requirement
ela = transcripts[transcripts['CSU Req'] == 'B']
ela

### Fixing BC

The BC courses have to be fixed because they represent a full year of coursework rather than one semester like the other courses do.

In [ ]:
# Non-BC courses are selected
non_bc = ela[ela['Crs Lvl'] != 24]
non_bc

In [ ]:
# BC courses selected
bc = ela[ela['Crs Lvl'] == 24]
bc

In [ ]:
# BC courses are duplicated
bc = pd.concat([bc, bc])
bc

In [ ]:
# The courses are all put back together
ela = pd.concat([bc, non_bc])
ela

### ELA GPA Calculation

The GPA will be calculated in the same manner as the Spanish GPA but it will only be for the top four courses that a student did.

In [ ]:
# Students are given credit for one semester for each course
ela['ELA Semesters'] = 1
ela

In [ ]:
# A grades are selected
ela_a = ela[(ela['Mark'] == 'A+') | (ela['Mark'] == 'A') | (ela['Mark'] == 'A-')]
ela_a

In [ ]:
# 4 points are given for each A
ela_a['Points'] = 4
ela_a

In [ ]:
# B grades are selected
ela_b = ela[(ela['Mark'] == 'B+') | (ela['Mark'] == 'B') | (ela['Mark'] == 'B-')]
ela_b

In [ ]:
# A score of 3 is given to each B grade
ela_b['Points'] = 3
ela_b

In [ ]:
# The C grades are selected
ela_c = ela[(ela['Mark'] == 'C+') | (ela['Mark'] == 'C') | (ela['Mark'] == 'C-')]
ela_c

In [ ]:
# A score of 2 is given for each C grade
ela_c['Points'] = 2
ela_c

In [ ]:
# The D grades are selected
ela_d = ela[(ela['Mark'] == 'D+') | (ela['Mark'] == 'D') | (ela['Mark'] == 'D-')]
ela_d

In [ ]:
# One point is given for each D
ela_d['Points'] = 1
ela_d

In [ ]:
# F grades are selected
ela_f = ela[ela['Mark'] == 'F']
ela_f

In [ ]:
# There is 0 points given for each F
ela_f['Points'] = 0
ela_f

In [ ]:
# All the grades are put back together again
ela = pd.concat([ela_a, ela_b, ela_c, ela_d, ela_f])
ela

In [ ]:
# Semesters completed by course are added
ela_courses = ela.groupby(['Student ID', 'Course'])['ELA Semesters'].sum().reset_index()
ela_courses

In [ ]:
# Points by course are added
ela_points = ela.groupby(['Student ID', 'Course'])['Points'].sum().reset_index()
ela_points

In [ ]:
# Semesters and Points are put together
ela = pd.merge(ela_courses, ela_points, how='inner', on=['Student ID', 'Course'])
ela

In [ ]:
# Sorts columns by student id and points in courses
ela = ela.sort_values(by=['Student ID', 'Points'], ascending=[False, False])
ela

In [ ]:
# Picks the top four courses
ela = ela.groupby(['Student ID']).head(4)
ela

In [ ]:
# Sums all of the semesters up
ela_courses = ela.groupby(['Student ID'])['ELA Semesters'].sum().reset_index()
ela_courses

In [ ]:
# Sums up the points earned in ELA
ela_points = ela.groupby(['Student ID'])['Points'].sum().reset_index()
ela_points

In [ ]:
# Merges semesters and points
ela = pd.merge(ela_courses, ela_points, how='inner', on='Student ID')
ela

In [ ]:
# Calculates a GPA by dividing points by semesters
ela['ELA GPA'] = ela['Points'] / ela['ELA Semesters']
ela

In [ ]:
# Narrows columns down to semesters and ELA GPA
ela = ela[['Student ID', 'ELA Semesters', 'ELA GPA']]
ela

## SBAC

How students performed on the SBAC will be looked at. It will be done by incoporating the cers file that is received and then narrowing it down to summative ELA. If a student scores a 3 or above they will earn a Met for this part if they do not get this score it will be noted that it is Not Met.

In [ ]:
tests

In [ ]:
# Chooses summative tests
tests = tests[tests['AssessmentType'] == 'Summative']
tests

In [ ]:
# Chooses ELA tests
tests = tests[tests['Subject'] == 'ELA']
tests

In [ ]:
# Narrows it down to subject and level
tests = tests[['StudentIdentifier', 'Subject', 'ScaleScoreAchievementLevel']]
tests

In [ ]:
# Selects levels 3 and above
sbacpass = tests[tests['ScaleScoreAchievementLevel'] >= 3]
sbacpass

In [ ]:
# Gives a value of Met for a new column called SBAC
sbacpass['SBAC'] = 'Met'
sbacpass

In [ ]:
# Selects students with a score of 2 or less
sbacfail = tests[tests['ScaleScoreAchievementLevel'] <= 2]
sbacfail

In [ ]:
# Gives Not Met for SBAC with the students
sbacfail['SBAC'] = 'Not Met'
sbacfail

In [ ]:
# Puts all of the SBAC scores back together
sbac = pd.concat([sbacpass, sbacfail])
sbac

## Merge

Rosters for students will be narrowed down to those that have taken Spanish 3 and then to those who are in the 12th grade. The data that has been worked on will all be merged with the filtered down roster.

In [ ]:
roster

In [ ]:
transcripts

In [ ]:
# Finds Spanish 3 among the transcripts of students
spanish3 = transcripts[transcripts['Course'] == 'Spanish 3']
spanish3

In [ ]:
# Selects only 12th graders that took Spanish 3
spanish3 = spanish3[spanish3['Grade'] == 12]
spanish3

In [ ]:
# Narrows down just to the Student ID
spanish3 = spanish3[['Student ID']]
spanish3

In [ ]:
# Does an inner join from the transcript to narrow down the roster to just these students
roster = pd.merge(roster, spanish3, how='inner', on='Student ID')
roster

In [ ]:
# Merges in the Spanish GPA
merge = pd.merge(roster, spanish, how='inner', on='Student ID')
merge

In [ ]:
# Merges in ELA GPA
merge = pd.merge(merge, ela, how='inner', on='Student ID')
merge

In [ ]:
# Merges in how students performed on the SBAC
merge = pd.merge(merge, sbac, how='left', left_on='State Student ID', right_on='StudentIdentifier')
merge

In [ ]:
# Drops any extra categories
merge = merge.drop(columns=['StudentIdentifier', 'Subject', 'ScaleScoreAchievementLevel'])
merge

## On Track

In [ ]:
merge

In [ ]:
# Students who have under a 3.0 in Spanish classes
spanish_no = merge[merge['Spanish GPA'] < 3]
spanish_no

In [ ]:
# Students are labeled as not on track since they are not meeting Spanish requirement
spanish_no['On Track'] = 'No'
spanish_no

In [ ]:
# Selecting students who have a Spanish GPA 3.0
spanish_yes = merge[merge['Spanish GPA'] >= 3]
spanish_yes

In [ ]:
# Selects students who have met SBAC who are meeting Spanish GPA requirement
spanish_sbac = spanish_yes[spanish_yes['SBAC'] == 'Met']
spanish_sbac

In [ ]:
# Saying these students are on track
spanish_sbac['On Track'] = 'Yes'
spanish_sbac

In [ ]:
# Selects students who did not meet in SBAC
spanish_no_sbac = spanish_yes[spanish_yes['SBAC'] != 'Met']
spanish_no_sbac

In [ ]:
# Selects students with above a 3.0 GPA in English
spanish_ela_yes = spanish_no_sbac[spanish_no_sbac['ELA GPA'] >= 3]
spanish_ela_yes

In [ ]:
# Marks these students On Track
spanish_ela_yes['On Track'] = 'Yes'
spanish_ela_yes

In [ ]:
# Marks students with under a 3.0 GPA as being that way
spanish_ela_no = spanish_no_sbac[spanish_no_sbac['ELA GPA'] < 3]
spanish_ela_no

In [ ]:
# These students are marked as not being on track
spanish_ela_no['On Track'] = 'No'
spanish_ela_no

In [ ]:
# Puts together all of the student data once again
final = pd.concat([spanish_sbac, spanish_ela_yes, spanish_ela_no, spanish_no])
final

## EL Students

Students that are English Language Learners have another criteria that needs to be met and that is for them to have a 4 on the Speaking and Listening part of the ELPAC. Using the data from the Aeries Query the part (Part 1) of the ELPAC will be chosen and then looked at to determine if these students are on track or not.

In [ ]:
final

In [ ]:
# Selects non-EL students
non_el = final[final['LangFlu'] != 'L']
non_el

In [ ]:
# Selects EL students
el = final[final['LangFlu'] == 'L']
el

In [ ]:
elpac

In [ ]:
# Selects ELPAC out of the tests
elpac = elpac[elpac['TESTID'] == 'ELPAC']
elpac

In [ ]:
# Searches for the right part of the test
elpac = elpac[elpac['Part'] == 1]
elpac

In [ ]:
elpac.info()

In [ ]:
# Changes date from a string into a datetime
elpac['Date Taken']= pd.to_datetime(elpac['Date Taken'])
elpac

In [ ]:
# Sorts by most recent date and down
elpac = elpac.sort_values(by=['Student ID' , 'Date Taken'], ascending=[False, False])
elpac

In [ ]:
# Drops all duplicates and keeps the most recent test.
elpac = elpac.drop_duplicates(subset='Student ID', keep='first')
elpac

In [ ]:
# Narrows columns down
elpac = elpac[['Student ID', 'PerformLvl']]
elpac

In [ ]:
# Renames columns
elpac = elpac.rename(columns={'PerformLvl': 'Oral Level'})
elpac

In [ ]:
el

In [ ]:
# Merges test with chosen EL students
el = pd.merge(el, elpac, how='inner', on='Student ID')
el

In [ ]:
el_on_track= el[el['On Track'] =='Yes']
el_on_track

In [ ]:
el_not_track= el[el['On Track'] =='No']
el_not_track

In [ ]:
passed_elpac = el_on_track[el_on_track['Oral Level'] >=4]
passed_elpac['On Track'] = 'Yes'
passed_elpac

In [ ]:
failed_elpac = el_on_track[el_on_track['Oral Level'] <3]
failed_elpac['On Track'] = 'No'
failed_elpac

In [ ]:
el = pd.concat([passed_elpac, failed_elpac, el_not_track])
el

In [ ]:
final = pd.concat([non_el, el])
final

In [ ]:
column_to_move = final.pop('On Track')
final['On Track'] = column_to_move
final

In [ ]:
import base64
from IPython.display import HTML

def create_download_link( df, title = "Biliteracy", filename = "Biliteracy"):
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(final)